In [1]:
##################################################################################
#                                                                                #
#                       Create / Train Networks to Verify                        #
#                                                                                #
##################################################################################

In [2]:
# =====================
# Imports
# =====================
import sys
import os
sys.path.append('mister_ed') # library for adversarial examples
import mnist.mnist_loader as  ml
from plnn import PLNN
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
from torch.autograd import Variable
from torchvision import datasets, transforms
MNIST_DIM = 784

/home/justin/.conda/envs/DeepL/lib/python3.7/site-packages/matplotlib/figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "


In [3]:
# Define functions to train and evaluate a network 

def l1_loss(net):
    return sum([_.norm(p=1) for _ in net.parameters() if _.dim() > 1])

def l2_loss(net):
    return sum([_.norm(p=2) for _ in net.parameters() if _.dim() > 1])

def train(net, trainset, num_epochs, min_acc, l1_reg, l1_scale=2e-3):
    opt = optim.Adam(net.parameters(), lr=1e-3, weight_decay=0)
    for epoch in range(num_epochs):
        err_acc = 0
        err_count = 0
        for data, labels in trainset:
            output = net(Variable(data.view(-1, 784)))
            l = nn.CrossEntropyLoss()(output, Variable(labels)).view([1])
            if l1_reg:
                l1_scale = torch.Tensor([l1_scale]).cuda()
                l += l1_scale * l1_loss(net).view([1])
            
            err_acc += (output.max(1)[1].data != labels).float().mean() 
            err_count += 1
            opt.zero_grad() 
            (l).backward() 
            opt.step()
        error = err_acc / err_count
        if epoch % 10 == 0 or epoch == num_epochs:
            print("(%02d) error:" % epoch, error)
        if error < (1-min_acc):
            break

        
def test_acc(net, valset):
    err_acc = 0 
    err_count = 0 
    for data, labels in valset:
        data, labels = (data.cuda(), labels.cuda())
        n = data.shape[0]
        output = net(Variable(data.view(-1, 784)))
        err_acc += (output.max(1)[1].data != labels).float().mean() * n
        err_count += n

    print("Test Accuracy of: %.03f" % (1 - (err_acc / err_count).item()))
    

In [4]:
##################################################################################
#                                                                                #
#                       Network Training                                         #
#                                                                                #
##################################################################################


l1_text = lambda reg_bool: '_l1_' if reg_bool else ''
hidden_layer_sizes_array = [[20, 20, 20], [20, 20]]
all_digits_array = [True, False]
l1_regularizations = [True, False]
l1_text = lambda reg_bool: '_l1_' if reg_bool else ''

for hidden_layer_sizes in hidden_layer_sizes_array:
    for ALL_DIGITS in all_digits_array:
        for reg in l1_regularizations:
            if not ALL_DIGITS:
                digits = [1, 7]
                l1_scale = 1e-3
                trainset = ml.load_single_digits('train', digits, batch_size=128, 
                                                  shuffle=False)
                trainset = [(data.cuda(), labels.cuda()) for (data, labels) in trainset]
                valset = ml.load_single_digits('val', digits, batch_size=128, 
                                                  shuffle=False)
                NETWORK_NAME = '17_mnist_'+l1_text(reg)+str(hidden_layer_sizes)+'.pkl'
                layer_sizes = [MNIST_DIM,]+hidden_layer_sizes+[2]
                num_epochs = 100
            else:
                l1_scale = 2e-3
                trainset = ml.load_mnist_data('train', batch_size=256, shuffle=False)
                trainset = [(data.cuda(), labels.cuda()) for (data, labels) in trainset]
                valset = ml.load_mnist_data('val', batch_size=256, shuffle=False)
                NETWORK_NAME = 'mnist_'+l1_text(reg)+str(hidden_layer_sizes)+'.pkl'
                layer_sizes = [MNIST_DIM,]+hidden_layer_sizes+[10]
                num_epochs = 150
                
            filepath = os.getcwd()+'/Models/'+NETWORK_NAME
            
            try:
                network = pickle.load(open(filepath, 'rb'))
                net = network.net
                print("Network Already Trained")
                print("Network:", NETWORK_NAME)
                print('Test Accuracy:', test_acc(net, valset))
            except:
                print("Training a new network")
                network = PLNN(layer_sizes)
                net = network.net.cuda()
                train(net, trainset, num_epochs, 0.98, reg, l1_scale)
                pickle.dump(network, open(filepath, 'wb'))
                print("Network:", NETWORK_NAME)
            print('============')


Training a new network
[784, 20, 20, 20, 10]
(00) error: tensor(0.7678, device='cuda:0')
(10) error: tensor(0.1710, device='cuda:0')
(20) error: tensor(0.1178, device='cuda:0')
(30) error: tensor(0.0994, device='cuda:0')
(40) error: tensor(0.0944, device='cuda:0')
(50) error: tensor(0.0923, device='cuda:0')
(60) error: tensor(0.0909, device='cuda:0')
(70) error: tensor(0.0906, device='cuda:0')
(80) error: tensor(0.0901, device='cuda:0')
(90) error: tensor(0.0891, device='cuda:0')
(100) error: tensor(0.0879, device='cuda:0')
(110) error: tensor(0.0872, device='cuda:0')
(120) error: tensor(0.0869, device='cuda:0')
(130) error: tensor(0.0862, device='cuda:0')
(140) error: tensor(0.0861, device='cuda:0')
Network: mnist__l1_[20, 20, 20].pkl
Accuracy of: 0.909
Accuracy: None
Training a new network
[784, 20, 20, 20, 10]
(00) error: tensor(0.3550, device='cuda:0')
(10) error: tensor(0.0474, device='cuda:0')
(20) error: tensor(0.0331, device='cuda:0')
(30) error: tensor(0.0265, device='cuda:0')